In [2]:
# ok I want with confidence, all zip codes reported by census, and their lat/lngs
import pandas as pd
import plotly.express as px
from IPython.display import display
pd.options.display.max_rows = 10
from sklearn.cluster import KMeans

from lib.census_helper import load_sweetened_zips

us_zips = load_sweetened_zips()
us_zips

,_zip,_lat,_lng,_census_total
0,47337,39.802754,-85.437285,203
1,47514,38.148600,-86.594016,1559
2,46045,40.289610,-86.148659,93
3,46704,41.103660,-85.292142,60
4,47585,38.268350,-87.158083,592
...,...,...,...,...
32918,37932,35.917299,-84.198787,14527
32919,37341,35.219931,-85.073003,12085
32920,37849,36.054050,-84.048488,25459
32921,37754,36.139099,-84.029801,4729


In [3]:
df = us_zips
km = KMeans(
    n_clusters=10, init='k-means++',
    n_init=10, max_iter=300, 
)
X = df[['_lat','_lng']]
y_km = km.fit_predict(X)
df['_y'] = y_km
df['y'] = df['_y'].apply(lambda x: f"group_{x}")

px.scatter(df, x='_lat', y='_lng', color='y')

elbow method evals performance

In [13]:
X = df[['_lat','_lng']]

distortions = []
K = range(1, 50)
for i in K:
    km = KMeans(
        n_clusters=i, init='k-means++',
        n_init=10, max_iter=300,
        tol=1e-04, random_state=0
    )
    km.fit(X)
    distortions.append(km.inertia_)

elbow_df = pd.DataFrame({'k': K, 'distortion': distortions})
px.scatter(elbow_df, x='k', y='distortion')

In [6]:
df.groupby(['_y']).agg(census_total=("_census_total","sum"))

,census_total
_y,
0,51973006
1,42498781
2,15451565
3,39638442
4,684972
5,19109293
6,49533892
7,49557699
8,1360301
